In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))


In [20]:
from langchain_ollama import ChatOllama
llm= ChatOllama(model="llama3.2:1b")

In [21]:
result.content

'It\'s difficult to pinpoint a single most famous food in any country, as different regions and cultures have their own beloved dishes. However, I can give you some insights based on popular opinions and culinary traditions.\n\nIn the United States, one of the most iconic and widely recognized foods is likely McDonald\'s Big Mac. Introduced in 1968, it consists of two all-beef patties, special sauce, lettuce, cheese, pickles, and onions on a sesame seed bun. Its widespread popularity and enduring presence in American cuisine make it a strong contender for this title.\n\nIn the United Kingdom, fish and chips are often considered one of the most famous foods in the country. Battered and fried fish, served with crispy fries (chips) and often accompanied by mushy peas, is a classic comfort food that has been enjoyed for generations.\n\nIn India, curries like chicken tikka masala and butter chicken are extremely popular dishes, particularly in urban areas. These flavorful and aromatic sauce

In [63]:
food_prompt = PromptTemplate(
    template="Please recommend the most famous food in the {country}. Return the name of the food only",
    input_variables=['country']
)
food_chain = food_prompt | llm | output_parser
food = food_chain.invoke({"country":"Japan"})
food

'Sushi'

In [86]:
recipe_prompt = PromptTemplate(
    template="Please tell me the {recipe} order of this food in detail",
    input_variables=['food']
)
recipe_chain = recipe_prompt | llm | output_parser
recipe = recipe_chain.invoke({"recipe": food})
recipe

'To place an order for sushi, you\'ll typically want to specify the type of sushi, any fillings or ingredients you\'d like, and any desired accompaniments. Here\'s a general guide on how to order sushi:\n\n1. **Type of Sushi**: You can ask for different types of sushi, such as:\n\t* Maki (rolls)\n\t* Nigiri (hand-formed sushi with raw or cooked seafood or vegetables)\n\t* Sashimi (raw seafood sliced into thin pieces)\n\t* Temaki (hand rolls)\n\t* Uramaki (inside-out rolls)\n2. **Fillings**: If you\'d like to add any fillings, specify them:\n\t* Raw fish (e.g., salmon, tuna, or yellowtail)\n\t* Cooked fish (e.g., shrimp, eel, or octopus)\n\t* Vegetables (e.g., cucumber, avocado, or carrot)\n\t* Egg\n3. **Desired Accompaniments**: You can ask for additional items to be served with your sushi:\n\t* Wasabi (a spicy condiment)\n\t* Soy sauce (a salty liquid)\n\t* Pickled ginger (a sweet and sour vegetable)\n\t* Sesame seeds or grated daikon radish\n4. **Size**: If you want a specific size o

In [71]:
final_chain= food_chain | recipe_chain

In [81]:
from langchain_core.runnables import RunnablePassthrough

final_chain = {"country": RunnablePassthrough()} | {"country": food_chain} | recipe_chain

In [83]:
result = final_chain.invoke({"country": "Japan"})
print(result)

KeyError: "Input to PromptTemplate is missing variables {'recipe'}.  Expected: ['recipe'] Received: ['country']\nNote: if you intended {recipe} to be part of the string and not a variable, please escape it with double curly braces like: '{{recipe}}'.\nFor troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/INVALID_PROMPT_INPUT "

In [84]:
prompt_template= PromptTemplate(
    template= "Please recommend the most famous food in the {country}. Return the name of the food only",
    input_variables= ['country']
)
output_parser= StrOutputParser() #AIMessage()를 Str 변환
output_parser.invoke(llm.invoke(prompt_template.invoke({'country':'Korea'})))

'Bibimbap'

In [87]:
recipe_prompt = PromptTemplate(
    template="Please tell me the recipe for {food} in detail",
    input_variables=['food']
)
recipe_output_parser = StrOutputParser()
recipe_output_parser.invoke(llm.invoke(recipe_prompt.invoke({'food': food})))

"Sushi is a traditional Japanese dish that consists of vinegared rice and various toppings, such as raw or cooked fish, vegetables, and seaweed, wrapped in seaweed sheets. Here's a detailed recipe for making sushi at home:\n\nIngredients:\n\nFor the sushi rice:\n\n* 1 cup Japanese short-grain rice (such as Koshihikari or Akita Komachi)\n* 1 3/4 cups water\n* 1 tablespoon rice vinegar\n* 1 teaspoon sugar\n* 1 teaspoon salt\n\nFor the fillings:\n\n* Choose one or a combination of the following:\n\t+ Raw fish: salmon, tuna, yellowtail, or any other type of fish you prefer (see notes)\n\t+ Cooked fish: eel, octopus, or any other type of cooked fish\n\t+ Vegetables: cucumber, carrot, avocado, or any other type of vegetable you like\n\t+ Seaweed: nori sheets (dried seaweed sheets)\n\nInstructions:\n\n**Step 1: Prepare the Sushi Rice**\n\n1. Rinse the rice thoroughly in a fine-mesh strainer under cold running water until the water runs clear.\n2. Drain the rice and place it in a medium saucep

In [111]:
final_chain = food_chain | recipe_chain
result= final_chain.invoke({"food","This country is very famous for its food"})

In [104]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
message_list=[
    SystemMessage(content="You are a helpful assistant!"), # 페르소나 부여
    
    HumanMessage(content=" What is the famous food in Korea?"), # 모범질문
    AIMessage(content="The famous food in Korea is Bibimbap"), #모법답안
    HumanMessage(content="What is the famous food of Japan?"), #모범 답안
    AIMessage(content="Japan's famous food is sushi"),
    HumanMessage(content="what is the famous food in USA")
]
result= llm.invoke(message_list)
result.content

"A question that's easy to answer! In the United States, some of the most famous foods include:\n\n1. Hamburgers: A classic American favorite made from ground beef, often served on a bun with lettuce, tomato, cheese, and condiments.\n2. Pizza: A popular Italian-American dish that originated in New York City, typically topped with tomato sauce, mozzarella cheese, and various meats or vegetables.\n3. Macaroni and Cheese: A comforting, creamy pasta dish that's a staple in many American households.\n4. Chicken Wings: Spicy or not, chicken wings are a popular snack or appetizer in the US, often served with a variety of sauces.\n5. BBQ Ribs: Slow-cooked pork ribs slathered in sweet and tangy barbecue sauce, typically grilled over an open flame.\n\nOf course, these are just a few examples, and there are many other delicious foods that are popular in the USA!"

In [112]:
print(result)

Here's a detailed description of sushi, including its components:

**Sushi Rolls**

A traditional sushi roll typically consists of three main components:

1. **Nori**: The seaweed sheet that wraps around the filling. Nori is made from dried and flattened seaweed sheets, usually from the kelp plant (Laminaria japonica). It's a crucial component of sushi, providing flavor, texture, and nutrition.
2. **Filling**: This can include various ingredients such as:
 * Raw or cooked fish (e.g., salmon, tuna, or yellowtail)
 * Cooked seafood (e.g., shrimp, crab, or eel)
 * Vegetables (e.g., cucumber, avocado, or carrot)
 * Egg
 * Cheese (e.g., cream cheese or mayonnaise)
3. **Rice**: Sushi rice is a short-grain Japanese rice that's prepared with rice vinegar, sugar, and salt. It's cooked to a sticky consistency and then mixed with sushi vinegar to give it flavor.

**Sushi Types**

There are several types of sushi rolls, including:

* **Maki**: Rolled sushi with nori and rice.
* **Nigiri**: Hand-fo